In [7]:
import pandas as pd

dates = pd.read_csv("./Dates.csv", names=["string_date", "std_date"])

In [8]:
dates

,string_date,std_date
0,"January 01, 2000",2000-01-01
1,"January 02, 2000",2000-01-02
2,"January 03, 2000",2000-01-03
3,"January 04, 2000",2000-01-04
4,"January 05, 2000",2000-01-05
...,...,...
1995,"September 22, 1992",1992-09-22
1996,"September 23, 1992",1992-09-23
1997,"September 24, 1992",1992-09-24
1998,"September 25, 1992",1992-09-25


In [14]:
dates["string_date"] = dates["string_date"].str.replace(',', '')
dates["std_date"] = dates["std_date"].str.replace('-', ' ')

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_hold, y_train, y_hold = train_test_split(dates["string_date"], dates["std_date"], test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_hold, y_hold, test_size=0.5, random_state=42)

In [17]:
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [10]:
vocab_size = 512
embed_size = 128

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, output_layer=output_layer)

final_outputs, final_state, final_sequence_lengths = decoder(decoder_embeddings,
                                                             initial_state=encoder_state,
                                                             sequence_length=sequence_lengths)

Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs, sequence_lengths], outputs=[Y_proba])